In [ ]:
# EfficientNet-b0 with 3-way split, AUC, Confusion Matrix
import os, numpy as np, torch, cv2
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import gc
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# --- 설정 ---
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 라벨 추출 ---
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3: return None
        return 1 if score >= 4 else 0
    except:
        return None

# --- 파일 로딩 및 3-way split ---
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)
train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

# --- Dataset 정의 ---
class LIDCDataset(Dataset):
    def __init__(self, file_paths, labels):
        self.file_paths = file_paths
        self.labels = labels

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img = np.load(self.file_paths[idx]).astype(np.float32)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, 0)
        return torch.tensor(img).float(), torch.tensor(self.labels[idx]).float()

# --- DataLoader ---
train_loader = DataLoader(LIDCDataset(train_files, train_labels), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(LIDCDataset(val_files, val_labels), batch_size=BATCH_SIZE)
test_loader = DataLoader(LIDCDataset(test_files, test_labels), batch_size=BATCH_SIZE)

# --- 모델 정의 ---
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
model = model.to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "best_model_efficientnet_b0.pth")
os.makedirs(os.path.dirname(save_path), exist_ok=True)
best_val_acc = 0.0

# --- 학습 루프 ---
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images, labels = images.to(DEVICE), labels.unsqueeze(1).to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}] Loss: {total_loss / len(train_loader):.4f}")

    gc.collect()
    torch.cuda.empty_cache()

    # --- 검증 ---
    model.eval()
    correct = total = 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            probs = torch.sigmoid(model(images)).squeeze()
            preds = (probs > 0.5).long()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)
    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print("✅ Best model saved!")

# --- 테스트 성능 평가 ---
print("\n📊 Test Set Evaluation (Best Model 기준):")
model.load_state_dict(torch.load(save_path))
model.eval()
y_true, y_pred, y_probs = [], [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        probs = torch.sigmoid(model(images)).squeeze()
        preds = (probs > 0.5).long()
        y_probs.extend(probs.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

test_acc = (np.array(y_pred) == np.array(y_true)).mean() * 100
print(f"✅ Test Accuracy: {test_acc:.2f}%")
print(classification_report(y_true, y_pred, digits=4))
try:
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC 계산 실패: 클래스가 모두 있어야 합니다.")
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

Epoch 1/100: 100%|██████████| 234/234 [00:11<00:00, 21.07it/s]


[Epoch 1] Loss: 0.6041
Validation Accuracy: 0.7262
✅ Best model saved!


Epoch 2/100: 100%|██████████| 234/234 [00:10<00:00, 21.99it/s]


[Epoch 2] Loss: 0.4676
Validation Accuracy: 0.7788
✅ Best model saved!


Epoch 3/100: 100%|██████████| 234/234 [00:11<00:00, 20.47it/s]


...

[Epoch 98] Loss: 0.0116
Validation Accuracy: 0.8662


Epoch 99/100: 100%|██████████| 234/234 [00:11<00:00, 20.93it/s]


[Epoch 99] Loss: 0.0127
Validation Accuracy: 0.8588


Epoch 100/100: 100%|██████████| 234/234 [00:10<00:00, 21.35it/s]


[Epoch 100] Loss: 0.0102
Validation Accuracy: 0.8738

📊 Test Set Evaluation (Best Model 기준):
✅ Test Accuracy: 85.12%
              precision    recall  f1-score   support

         0.0     0.7727    0.8036    0.7879       275
         1.0     0.8949    0.8762    0.8855       525

    accuracy                         0.8512       800
   macro avg     0.8338    0.8399    0.8367       800
weighted avg     0.8529    0.8512    0.8519       800

AUC: 0.9049
Confusion Matrix:
[[221  54]
 [ 65 460]]
